This code covers the fighter career statistics database. Data web scraped from http://www.ufcstats.com/statistics/events/completed as well as https://www.ufc.com/rankings.

## Webscraping and dataframe creation

In the first section I will gather all fighter details from http://www.ufcstats.com/statistics/fighters, since it is done alphabetically I create a list containing each letter from the alphabet. Then use the first letter (a) as the starting dataframe and append  upon iteration of website links containing fighter details.

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# creating the alphabet for each letter of fighter last names, excluding a
al = ['b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [4]:
# setting the letter a as the first entry
data = pd.read_html('http://ufcstats.com/statistics/fighters?char=a&page=all')[0]
data.head(3)

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN


In [5]:
# Going through each link of fighter last names, then adding to the exisiting dataset

for i in range(len(al)):
    temp = pd.read_html('http://ufcstats.com/statistics/fighters?char=' + al[i] + '&page=all')[0]
    data = pd.concat([data,temp])

In [6]:
# Dropping columns with all nan values
data.dropna(how = 'all', inplace=True)

Next I will add the current UFC rankings from the offical website. 

In [7]:
# All weight class names
weights = ["Pound4Pound", "Flyweight", "Bantamweight", "Featherweight", "Lightweight", "Welterweight",
           "Middleweight", "Light Heabyweight", "Heavyweight", "Women's Strawweight", "Women's Flyweight",
           "Women's Bantamweight"]


rankings = pd.read_html("https://www.ufc.com/rankings")[0]
rankings["Weightclass"] = "LB4LB"


for i in range(1,len(weights)): 
    temp = pd.read_html("https://www.ufc.com/rankings")[i]
    temp["Weightclass"] = weights[i]
    rankings = rankings.append(temp)
    
rankings.head(3)

,0,1,2,Weightclass
0,2,Jon Jones,NaN,LB4LB
1,3,Israel Adesanya,NaN,LB4LB
2,4,Stipe Miocic,NaN,LB4LB


## Advanced Statistic Additions

The next stage is to create new features and advanced statistics to the fighter details dataset. Used with the advanced dataset which contains every fight that has occurred in the sport. To do this we connect first through the postgre SQL Database

In [4]:
import psycopg2
import requests

con = psycopg2.connect("Connection String")
print("Database opened successfully")
print()
cur = con.cursor()
cur.execute('SELECT * from "Every_Fight"')
tupples = cur.fetchall()
headers = [i[0] for i in cur.description]
# Dataframe created from tupples and headers
advanced = pd.DataFrame(tupples, columns=headers)


advanced.tail(3) 

Database opened successfully



,Unnamed: 0,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,REV_RED,REV_BLUE,CONTROL_RED,CONTROL_BLUE,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND1_CONTROL_RED,RND1_CONTROL_BLUE,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND2_CONTROL_RED,RND2_CONTROL_BLUE,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND3_CONTROL_RED,RND3_CONTROL_BLUE,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND4_CONTROL_RED,RND4_CONTROL_BLUE,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_REV_RED,RND5_REV_BLUE,RND5_CONTROL_RED,RND5_CONTROL_BLUE,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE,LOSER
5921,9,Tafon Nchukwi,Jamie Pickett,Middleweight Bout,UFC Fight Night: Thompson vs. Neal,"December 19, 2020","Las Vegas, Nevada, USA",0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Chris Tognoni,Eric Colon25 - 30.Mike Bell26 - 30.Chris Lee26...,Tafon Nchukwi,Non-Title Bout,,1.0,0.0,120 of 187,40 of 81,64%,49%,159 of 234,40 of 81,0 of 2,1 of 4,0%,25%,0.0,0.0,0

In [9]:
fi = data

In [10]:
advanced.iloc[:,1:].head(3)

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,REV_RED,REV_BLUE,CONTROL_RED,CONTROL_BLUE,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND1_CONTROL_RED,RND1_CONTROL_BLUE,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND2_CONTROL_RED,RND2_CONTROL_BLUE,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND3_CONTROL_RED,RND3_CONTROL_BLUE,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND4_CONTROL_RED,RND4_CONTROL_BLUE,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_REV_RED,RND5_REV_BLUE,RND5_CONTROL_RED,RND5_CONTROL_BLUE,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE,LOSER
0,Gerard Gordea,Teila Tul,Open Weight Bout,UFC 1: The Beginning,1993-11-12,"Denver, Colorado, USA","2,800",KO/TKO,1,0:26,No Time Limit,Joao Alberto Barreto,Kick to Head On Ground,Gerard Gordeau,Non-Title Bout,,0.0,0.0,3 of 5,0 of 1,60%,0%,3 of 5,0 of 1,0 of 0,0 of 1,0%,0%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3 of 5,0 of 1,60%,0%,3 of 5,0 of 1,0 of 0,0 of 1,0%,0%,0.0,0.0,

In [11]:
advanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5924 entries, 0 to 5923
Columns: 222 entries, Unnamed: 0 to LOSER
dtypes: float64(33), int64(2), object(187)
memory usage: 10.0+ MB


In [12]:
# Create a column for full name
fi['Full'] = fi['First'] + ' ' + fi['Last']
fi.head(3)

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,Full
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN,Tom Aaron
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN,Danny Abbadi
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN,David Abbott


After web-scraping the UFC fight data, I noticed that some names had clerical errors. In some cases names were missing the last letter, for example “Brock” might be recorded as “Broc” in some instances. This function is used to check for this. 

In [13]:
def name_check(name,check):
    """
    -------------------------------------------------------
    Checks to see if two names are the same despite errors
    in recording
    Use: result = name_check(name,check)
    -------------------------------------------------------
    Parameters:
        name - the proper full name (str)
        check - the name tested for comparison (str)
    Returns:
        Result - Whether the names match or not (bool)
    -------------------------------------------------------
    """
    try:
        name = name.lower()
        check = check.lower()
        
    except:
        return False
    # If the name length is more than 1 it has to be false
    
    if abs(len(name) - len(check)) > 1:
        result = False
        
    # If the name is in check
        
    elif name in check or check in name:
        result = True
        
    else:
        result = False
        
    return result

For most of the calculations the fighter name record from the fi dataframe is iterated through the advanced (fight database). After this the results are added to the fighter details (fi) dataset for that specific fighter. 

Some common referred to acronyms include:

- Red - Red corner
- Blue - Blue corner
- Sig - Significant strikes 
- Eaten - Absorbed strikes 
- TD - Take down
- Sub - Submission 
- KO - Knockout
- Dec - Decision

Lets add the current ranking and the highest ranking the fighter has every been in his or her career.

In [ ]:
# Creating UFC records for all fighters that have fought in the ufc

fi['UFC_W']= 0 
fi['UFC_L']= 0 
fi['UFC_D']= 0
fi['UFC_NC'] = 0


count = 0
for i in fi['Full']:
    for j in advanced['LOSER']:
        if name_check(i,j):
            fi['UFC_L'].iloc[count] += 1
            
    count += 1 
    
    
count = 0
for i in fi['Full']:
    for j in advanced[['WINNER','FIGHTER_RED','FIGHER_BLUE','FINISH']].values:
        if len(j[0]) > 1 and len(i) > 1:
            if name_check(i,j[0]):
                fi['UFC_W'].iloc[count] += 1
                
            elif (name_check(i,j[1]) or name_check(i,j[2])) and j[0] == 'Draw':
                if j[3] == 'Overturned' or 'Could' in j[3]:
                    fi["UFC_NC"].iloc[count] += 1

                else:
                    fi['UFC_D'].iloc[count] += 1
                    print(j)
                
            
    count += 1

In [15]:
#The dataset includes fighters from strikeforce (not apart of the UFC), therefore they have 0 fights and are removed
fi['FIGHTS'] = fi['UFC_W'] + fi['UFC_L'] + fi['UFC_D']
fi2 = fi[fi['FIGHTS'] != 0]
fi2.reset_index(inplace = True)
fi2.head(3)

,index,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,Full,UFC_W,UFC_L,UFC_D,UFC_NC,FIGHTS
0,2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN,Danny Abbadi,0,2,0,0,2
1,3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN,David Abbott,8,10,0,0,18
2,4,Shamil,Abdurakhimov,Abrek,"6' 3""",235 lbs.,"76.0""",Orthodox,20.0,5.0,0.0,NaN,Shamil Abdurakhimov,5,3,0,0,8


In [ ]:
# Adding total strikes and sigificant strikes landed and absorbed for each fighter.


#offense
fi2['SIG_LAND'] = 0
fi2['SIG_THROWN'] = 0
fi2['TOTAL_LAND'] = 0
fi2['TOTAL_THROWN'] = 0

#Defense
fi2['TOTAL_EATEN_L'] = 0
fi2['TOTAL_EATEN_T'] = 0
fi2['SIG_EATEN_L'] = 0
fi2['SIG_EATEN_T'] = 0


# itterate through each fighter and every fight, if theres a match add to the strike total
count = 0 
for i in fi2['Full']:
    for a in advanced[['FIGHTER_RED','TOTAL_STR_RED','SIG_STR_RED','FIGHER_BLUE','TOTAL_STR_BLUE','SIG_STR_BLUE']].values:
        
        if name_check(i,a[0]):
            if a[1] != 0 and a[1] != '0':
                
                
                # columns are all formatted as "n1 of n2"
                t = a[1].split(' of ')
                s = a[2].split(' of ')
               
                
            
                fi2['TOTAL_LAND'].iloc[count] += int(t[0])
                fi2['TOTAL_THROWN'].iloc[count] += int(t[1])
                fi2['SIG_LAND'].iloc[count] += int(s[0])
                fi2['SIG_THROWN'].iloc[count] += int(s[1])
                
                
                
                # The other strike columns are the strikes the fighter has absorbed
                
                t = a[4].split(' of ')
                s = a[5].split(' of ')
                
                
            
                fi2['TOTAL_EATEN_L'].iloc[count] += int(t[0])
                fi2['TOTAL_EATEN_T'].iloc[count] += int(t[1])
                fi2['SIG_EATEN_L'].iloc[count] += int(s[0])
                fi2['SIG_EATEN_T'].iloc[count] += int(s[1])
                
        elif name_check(i,a[3]):
            if a[4] != 0 and a[4] != '0':
                t = a[4].split(' of ')
                s = a[5].split(' of ')
                
                print(a)
        
                fi2['TOTAL_LAND'].iloc[count] += int(t[0])
                fi2['TOTAL_THROWN'].iloc[count] += int(t[1])
                fi2['SIG_LAND'].iloc[count] += int(s[0])
                fi2['SIG_THROWN'].iloc[count] += int(s[1])
                
                
                
                
                t = a[1].split(' of ')
                s = a[2].split(' of ')
                
                
            
                fi2['TOTAL_EATEN_L'].iloc[count] += int(t[0])
                fi2['TOTAL_EATEN_T'].iloc[count] += int(t[1])
                fi2['SIG_EATEN_L'].iloc[count] += int(s[0])
                fi2['SIG_EATEN_T'].iloc[count] += int(s[1])
            
    count += 1

In [17]:
# accuracy percentages both defense and offense

fi2['SIG_PER'] = round(fi2['SIG_LAND'] / fi2['SIG_THROWN'] * 100,2)
fi2['TOTAL_PER'] = round(fi2['TOTAL_LAND']/fi2['TOTAL_THROWN'] * 100,2)


In [18]:
# Takedowns, sub attempts, passes and reversals for every fighter
fi2['TD_L'] = 0
fi2['TD_A'] =0
fi2['SUB_A']=0
fi2['REV']=0


count = 0 
for i in fi2['Full']:
    for a in advanced[['FIGHTER_RED','FIGHER_BLUE','TD_RED','TD_BLUE','SUB_ATT_RED','SUB_ATT_BLUE','REV_RED','REV_BLUE']].values:
    
        if name_check(i,a[0]):
            if a[2] != 0 and a[2] != '0':
                t = a[2].split(' of ')
                s = a[4]
                r = a[6]
                
           
                fi2['TD_L'].iloc[count] += int(t[0])
                fi2['TD_A'].iloc[count] += int(t[1])
                fi2['SUB_A'].iloc[count] += int(s)
                
                fi2['REV'].iloc[count] += int(r)
                
        elif name_check(i,a[1]):
            if a[3] != 0 and a[3] != '0':
                t = a[3].split(' of ')
                s = a[5]
                r= a[7]
            
                
                fi2['TD_L'].iloc[count] += int(t[0])
                fi2['TD_A'].iloc[count] += int(t[1])
                fi2['SUB_A'].iloc[count] += int(s)
                
                fi2['REV'].iloc[count] += int(r)
        
            
    count += 1

In [19]:
# takedown percetage calculation
fi2['TD_PERC'] = round((fi2['TD_L'] / fi2['TD_A'] * 100),2)

In [20]:
# Add KO , SUB and decision wins (method of victories)

fi2['KOs'] = 0
fi2['SUBS'] =0
fi2['DEC']=0


count = 0 
for i in fi2['Full']:
    for a in advanced[['FINISH','WINNER']].values:
        if name_check(i,a[1]):
            if 'Decision' in a[0]:
                fi2['DEC'].iloc[count] += 1
            elif 'KO' in a[0]:
                fi2['KOs'].iloc[count] += 1
            
            elif 'Sub' in a[0]:
                fi2['SUBS'].iloc[count] += 1
                
                      
        
            
    count += 1

In this block I create strike numbers for each athletes career. The logic is that if the fighter is in the red corner his offensive numbers are red corner labelled columns. As his defensive numbers are the blue corner labelled columns. Vice versa for fighters out of the blue corner. 

In [21]:
# Adding strikes absorbed and landed for every round

#OFFENSE
fi2['RD1_STRIKES_T'] = 0
fi2['RD1_STRIKES_L'] = 0
fi2['RD2_STRIKES_T'] = 0
fi2['RD2_STRIKES_L'] = 0
fi2['RD3_STRIKES_T'] = 0
fi2['RD3_STRIKES_L'] = 0
fi2['RD4_STRIKES_T'] = 0
fi2['RD4_STRIKES_L'] = 0
fi2['RD5_STRIKES_T'] = 0
fi2['RD5_STRIKES_L'] = 0

#DEFENSE
fi2['EATEN_RD1_STRIKES_T'] = 0
fi2['EATEN_RD1_STRIKES_L'] = 0
fi2['EATEN_RD2_STRIKES_T'] = 0
fi2['EATEN_RD2_STRIKES_L'] = 0
fi2['EATEN_RD3_STRIKES_T'] = 0
fi2['EATEN_RD3_STRIKES_L'] = 0
fi2['EATEN_RD4_STRIKES_T'] = 0
fi2['EATEN_RD4_STRIKES_L'] = 0
fi2['EATEN_RD5_STRIKES_T'] = 0
fi2['EATEN_RD5_STRIKES_L'] = 0

count = 0 
for i in fi2['Full']:
    for a in advanced[['FIGHTER_RED','FIGHER_BLUE', 'RND1_TOTAL_STR_RED', 'RND1_TOTAL_STR_BLUE', 'RND2_TOTAL_STR_RED', 'RND2_TOTAL_STR_BLUE','RND3_TOTAL_STR_RED', 'RND3_TOTAL_STR_BLUE', 'RND4_TOTAL_STR_RED', 'RND4_TOTAL_STR_BLUE','RND5_TOTAL_STR_RED','RND5_TOTAL_STR_BLUE']].values:
    
        if name_check(i,a[0]):
            if a[2] not in ['0',0,0.0,'0.0'] :
                
                t = a[2].split(' of ')
                
                fi2['RD1_STRIKES_L'].iloc[count] += int(t[0])
                fi2['RD1_STRIKES_T'].iloc[count] += int(t[1])
                
                # defense
                
                t = a[3].split(' of ')
                
                fi2['EATEN_RD1_STRIKES_L'].iloc[count] += int(t[0])
                fi2['EATEN_RD1_STRIKES_T'].iloc[count] += int(t[1])
                
                
                if a[4] not in ['0',0,0.0,'0.0']:
                                        

                    t = a[4].split(' of ')
                    
                    
                    fi2['RD2_STRIKES_L'].iloc[count] += int(t[0])
                    fi2['RD2_STRIKES_T'].iloc[count] += int(t[1])
                    
                    # defense
                
                    t = a[5].split(' of ')
                
                    fi2['EATEN_RD2_STRIKES_L'].iloc[count] += int(t[0])
                    fi2['EATEN_RD2_STRIKES_T'].iloc[count] += int(t[1])
                    
                    
                    if a[6] not in ['0',0,0.0,'0.0']:
                        t = a[6].split(' of ')
                    
                        fi2['RD3_STRIKES_L'].iloc[count] += int(t[0])
                        fi2['RD3_STRIKES_T'].iloc[count] += int(t[1])
                        
                        # defense
                
                        t = a[7].split(' of ')
                        
                
                        fi2['EATEN_RD3_STRIKES_L'].iloc[count] += int(t[0])
                        fi2['EATEN_RD3_STRIKES_T'].iloc[count] += int(t[1])
                        
                        if a[8] not in ['0',0,0.0,'0.0']:
                            t = a[8].split(' of ')
                    
                            fi2['RD4_STRIKES_L'].iloc[count] += int(t[0])
                            fi2['RD4_STRIKES_T'].iloc[count] += int(t[1])
                            
                            # defense
                
                            t = a[9].split(' of ')
            
                            fi2['EATEN_RD4_STRIKES_L'].iloc[count] += int(t[0])
                            fi2['EATEN_RD4_STRIKES_T'].iloc[count] += int(t[1])
                        
                            
                            if a[10] not in ['0',0,0.0,'0.0']:
                                t = a[10].split(' of ')
                    
                                fi2['RD5_STRIKES_L'].iloc[count] += int(t[0])
                                fi2['RD5_STRIKES_T'].iloc[count] += int(t[1])
                            
                                # defense
                
                                t = a[11].split(' of ')
                                
                    
                                fi2['EATEN_RD5_STRIKES_L'].iloc[count] += int(t[0])
                                fi2['EATEN_RD5_STRIKES_T'].iloc[count] += int(t[1])
                                                                

                                                                

                    
                
                
        elif name_check(i,a[1]):
            
            if a[2] not in ['0',0,0.0,'0.0']: 
                t = a[3].split(' of ')
                
                fi2['RD1_STRIKES_L'].iloc[count] += int(t[0])
                fi2['RD1_STRIKES_T'].iloc[count] += int(t[1])
                
                # defense
                
                t = a[2].split(' of ')

                fi2['EATEN_RD1_STRIKES_L'].iloc[count] += int(t[0])
                fi2['EATEN_RD1_STRIKES_T'].iloc[count] += int(t[1])
                
                if a[5] not in ['0',0,0.0,'0.0'] :                                        

                    t = a[5].split(' of ')
                    
                    fi2['RD2_STRIKES_L'].iloc[count] += int(t[0])
                    fi2['RD2_STRIKES_T'].iloc[count] += int(t[1])
                    
                     # defense
                
                    t = a[4].split(' of ')
                
                    fi2['EATEN_RD2_STRIKES_L'].iloc[count] += int(t[0])
                    fi2['EATEN_RD2_STRIKES_T'].iloc[count] += int(t[1])
                    
                    if a[7] not in ['0',0,0.0,'0.0'] :
                        t = a[7].split(' of ')
                    
                        fi2['RD3_STRIKES_L'].iloc[count] += int(t[0])
                        fi2['RD3_STRIKES_T'].iloc[count] += int(t[1])
                        
                        # defense
                
                        t = a[6].split(' of ')
                    
                
                        fi2['EATEN_RD3_STRIKES_L'].iloc[count] += int(t[0])
                        fi2['EATEN_RD3_STRIKES_T'].iloc[count] += int(t[1])
                        
                        if a[9] not in ['0',0,0.0,'0.0'] :
                           
                            t = a[9].split(' of ')
                            
                        
                    
                            fi2['RD4_STRIKES_L'].iloc[count] += int(t[0])
                            fi2['RD4_STRIKES_T'].iloc[count] += int(t[1])
                            
                            # defense
                
                            t = a[8].split(' of ')
                
                
                            fi2['EATEN_RD4_STRIKES_L'].iloc[count] += int(t[0])
                            fi2['EATEN_RD4_STRIKES_T'].iloc[count] += int(t[1])
                            
                            if a[11] not in ['0',0,0.0,'0.0']:
                              
                                t = a[11].split(' of ')
                    
                                fi2['RD5_STRIKES_L'].iloc[count] += int(t[0])
                                fi2['RD5_STRIKES_T'].iloc[count] += int(t[1])
                            
                            
                                 # defense
                
                                t = a[10].split(' of ')
                
                                fi2['EATEN_RD5_STRIKES_L'].iloc[count] += int(t[0])
                                fi2['EATEN_RD5_STRIKES_T'].iloc[count] += int(t[1])
        
            
    count += 1

In [22]:
fi = fi2

In [23]:
# Changing weight to a integer
count = 0
for i in fi['Wt.']:
    if i != '--':
        fi['Wt.'].loc[count] = int(i[0:3])
    
    
    count += 1

In [ ]:
# adding fight time and split decisions on each fighters record
fi['FIGHT_TIME'] = 0
fi['SPLIT_DEC']=0
fi['SPLIT_W']=0
fi['SPLIT_L']=0
fi['MAJ_W'] = 0
fi['MAJ_L'] = 0
fi['MAJ_DEC'] = 0

count = 0
for i in fi['Full']:
    for a in advanced[['FIGHTER_RED','FIGHER_BLUE','ROUND_FINISHED','TIME_FINISHED','FINISH', 'WINNER']].values:
        if name_check(i,a[0]):
            rd = (float(a[2])-1)*5
            miniute = float(a[3][0])
            try:
                second = float(a[3][2:])/100
                
            except:
                second = 0
            
            time = rd + miniute + second
            fi['FIGHT_TIME'].loc[count] += time
            
            if 'plit' in a[4] and 'raw' not in a[5]:
                fi['SPLIT_DEC'].loc[count] += 1
            
            if 'plit' in a[4] and name_check(i,a[5]):
                fi['SPLIT_W'].loc[count] += 1
            
            elif 'plit' in a[4] and 'raw' not in a[5]:
                fi['SPLIT_L'].loc[count] += 1
                
            
            if 'ajority' in a[4] and 'raw' not in a[5]:
                fi['MAJ_DEC'].loc[count] += 1
            
            if'ajority' in a[4] and name_check(i,a[5]):
                fi['MAJ_W'].loc[count] += 1
            
            elif 'ajority' in a[4] and 'raw' not in a[5]:
                fi['MAJ_L'].loc[count] += 1
            
        elif name_check(i,a[1]):
            rd = (float(a[2])-1)*5
            miniute = float(a[3][0])
            try:
                second = float(a[3][2:])/100
                
            except:
                second = 0
            time = rd + miniute + second
            fi['FIGHT_TIME'].loc[count] += time
            
            if 'plit' in a[4] and 'raw' not in a[5]:
                    fi['SPLIT_DEC'].loc[count] += 1
                    
            if 'plit' in a[4] and name_check(i,a[5]):
                fi['SPLIT_W'].loc[count] += 1
            
            elif 'plit' in a[4] and 'raw' not in a[5]:
                fi['SPLIT_L'].loc[count] += 1
                
            if 'ajority' in a[4] and 'raw' not in a[5]:
                print(i)
                fi['MAJ_DEC'].loc[count] += 1
            
            if'ajority' in a[4] and name_check(i,a[5]):
                fi['MAJ_W'].loc[count] += 1
            
            elif 'ajority' in a[4] and 'raw' not in a[5]:
                fi['MAJ_L'].loc[count] += 1
                    
                    
    count += 1      

In [25]:
#adding round by round percentages

fi['RD1_STR_PERC'] = round(fi['RD1_STRIKES_L'] / fi['RD1_STRIKES_T'],2)
fi['RD2_STR_PERC'] = round(fi['RD2_STRIKES_L'] / fi['RD2_STRIKES_T'],2)
fi['RD3_STR_PERC'] = round(fi['RD3_STRIKES_L'] / fi['RD3_STRIKES_T'],2)
fi['RD4_STR_PERC'] = round(fi['RD4_STRIKES_L'] / fi['RD4_STRIKES_T'],2)
fi['RD5_STR_PERC'] = round(fi['RD5_STRIKES_L'] / fi['RD5_STRIKES_T'],2)
fi['SIG_PER_MIN'] = round(fi['SIG_LAND'] / fi['FIGHT_TIME'],2)
fi['TOTAL_PER_MIN'] = round(fi['TOTAL_LAND'] / fi['FIGHT_TIME'],2)

fi.head(3)

,index,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,Full,UFC_W,UFC_L,UFC_D,UFC_NC,FIGHTS,SIG_LAND,SIG_THROWN,TOTAL_LAND,TOTAL_THROWN,TOTAL_EATEN_L,TOTAL_EATEN_T,SIG_EATEN_L,SIG_EATEN_T,SIG_PER,TOTAL_PER,TD_L,TD_A,SUB_A,REV,TD_PERC,KOs,SUBS,DEC,RD1_STRIKES_T,RD1_STRIKES_L,RD2_STRIKES_T,RD2_STRIKES_L,RD3_STRIKES_T,RD3_STRIKES_L,RD4_STRIKES_T,RD4_STRIKES_L,RD5_STRIKES_T,RD5_STRIKES_L,EATEN_RD1_STRIKES_T,EATEN_RD1_STRIKES_L,EATEN_RD2_STRIKES_T,EATEN_RD2_STRIKES_L,EATEN_RD3_STRIKES_T,EATEN_RD3_STRIKES_L,EATEN_RD4_STRIKES_T,EATEN_RD4_STRIKES_L,EATEN_RD5_STRIKES_T,EATEN_RD5_STRIKES_L,FIGHT_TIME,SPLIT_DEC,SPLIT_W,SPLIT_L,MAJ_W,MAJ_L,MAJ_DEC,RD1_STR_PERC,RD2_STR_PERC,RD3_STR_PERC,RD4_STR_PERC,RD5_STR_PERC,SIG_PER_MIN,TOTAL_PER_MIN
0,2,Danny,Abbadi,The Assassin,"5' 11""",155,--,Orthodox,4.0,6.0,0.0,NaN,Danny Abbadi,0,2,0,0,2,59,155,85,185,126,241,79,188,38.06,45.95,0,0,0,0,NaN,0,0,0,55,33,52,22,78,30,0,0,0,0,100,76,80,32,61,18,0,0,0,0,17.56,1,0,1,0,0,0,0.60,0.42,0.38,NaN,NaN,3.36,4.84
1,3,David,Abbott,Tank,"6' 0""",265,--,Switch,10.0,15.0,0.0,NaN,David Abbott,8,10,0,0,18,156,351,311,533,654,781,209,308,44.44,58.35,10,16,4,10,62.50,5,2,1,490,283,43,28,0,0,0,0,0,0,731,617,50,37,0,0,0,0,0,0,48.98,0,0,0,0,0,0,0.58,0.65,NaN,NaN,NaN,3.18,6.35
2,4,Shamil,Abdurakhimov,Abrek,"6' 3""",235,"76.0""",Orthodox,20.0,5.0,0.0,NaN,Shamil Abdurakhimov,5,3,0,0,8,210,469,341,613,269,584,210,508,44.78,55.63,7,29,1,4,24.14,2,0,3,252,133,203,129,149,76,9,3,0,0,261,118,151,71,124,52,48,28,0,0,84.60,1,1,0,0,0,0,0.53,0.64,0.51,0.33,NaN,2.48,4.03


In [26]:
# offensive and defensive numbers for all kinds of strike locations

# OFFENSE

fi['HEAD_STR_L'] = 0
fi['HEAD_STR_T'] = 0
fi['BODY_STR_L']=0
fi['BODY_STR_T']=0
fi['LEG_STR_L'] = 0
fi['LEG_STR_T'] = 0
fi['DISTANCE_STR_L']=0
fi['DISTANCE_STR_T']=0
fi['CLINCH_STR_L'] = 0
fi['CLINCH_STR_T'] = 0
fi['GROUND_STR_L'] = 0
fi['GROUND_STR_T'] = 0

#DEFENSE

fi['EATEN_HEAD_STR_L'] = 0
fi['EATEN_HEAD_STR_T'] = 0
fi['EATEN_BODY_STR_L']=0
fi['EATEN_BODY_STR_T']=0
fi['EATEN_LEG_STR_L'] = 0
fi['EATEN_LEG_STR_T'] = 0
fi['EATEN_DISTANCE_STR_L']=0
fi['EATEN_DISTANCE_STR_T']=0
fi['EATEN_CLINCH_STR_L'] = 0
fi['EATEN_CLINCH_STR_T'] = 0
fi['EATEN_GROUND_STR_L'] = 0
fi['EATEN_GROUND_STR_T'] = 0


count = 0
for i in fi['Full']:
    for a in advanced[['FIGHTER_RED','FIGHER_BLUE','HEAD_STR_RED','HEAD_STR_BLUE','BODY_STR_RED','BODY_STR_BLUE','LEG_STR_RED','LEG_STR_BLUE','DISTANCE_RED','DISTANCE_BLUE','CLINCH_RED','CLICH_BLUE','GROUD_RED','GROUND_BLUE']].values:
        if name_check(i,a[0]):
            if a[2] not in ['0',0,0.0,'None',None]:
                #OFFENSE
                h = a[2].split(' of ')
                b = a[4].split(' of ')
                l = a[6].split(' of ')
                d = a[8].split(' of ')
                c = a[10].split(' of ')
                g = a[12].split(' of ')
                
            
               
                fi['HEAD_STR_L'].iloc[count] += int(h[0])
                fi['HEAD_STR_T'].iloc[count] += int(h[1])
                fi['BODY_STR_L'].iloc[count] += int(b[0])
                fi['BODY_STR_T'].iloc[count] += int(b[1])
                fi['LEG_STR_L'].iloc[count] += int(l[0])
                fi['LEG_STR_T'].iloc[count] += int(l[1])
                fi['DISTANCE_STR_L'].iloc[count] += int(d[0])
                fi['DISTANCE_STR_T'].iloc[count] += int(d[1])
                fi['CLINCH_STR_L'].iloc[count] += int(c[0])
                fi['CLINCH_STR_T'].iloc[count] += int(c[1])
                fi['GROUND_STR_L'].iloc[count] += int(g[0])
                fi['GROUND_STR_T'].iloc[count] += int(g[1])
                
                
                
                # DEFNESE
                hd = a[3].split(' of ')
                bd = a[5].split(' of ')
                ld = a[7].split(' of ')
                dd = a[9].split(' of ')
                cd = a[11].split(' of ')
                gd = a[13].split(' of ')
            
            
                
               
                fi['EATEN_HEAD_STR_L'].iloc[count] += int(hd[0])
                fi['EATEN_HEAD_STR_T'].iloc[count] += int(hd[1])
                fi['EATEN_BODY_STR_L'].iloc[count] += int(bd[0])
                fi['EATEN_BODY_STR_T'].iloc[count] += int(bd[1])
                fi['EATEN_LEG_STR_L'].iloc[count] += int(ld[0])
                fi['EATEN_LEG_STR_T'].iloc[count] += int(ld[1])
                fi['EATEN_DISTANCE_STR_L'].iloc[count] += int(dd[0])
                fi['EATEN_DISTANCE_STR_T'].iloc[count] += int(dd[1])
                fi['EATEN_CLINCH_STR_L'].iloc[count] += int(cd[0])
                fi['EATEN_CLINCH_STR_T'].iloc[count] += int(cd[1])
                fi['EATEN_GROUND_STR_L'].iloc[count] += int(gd[0])
                fi['EATEN_GROUND_STR_T'].iloc[count] += int(gd[1])
                
        elif name_check(i,a[1]):
            if a[3] not in ['0',0,0.0,'None',None]:
                
                # Offense
                h = a[3].split(' of ')
                b = a[5].split(' of ')
                l = a[7].split(' of ')
                d = a[9].split(' of ')
                c = a[11].split(' of ')
                g = a[13].split(' of ')
                
                
                fi['HEAD_STR_L'].iloc[count] += int(h[0])
                fi['HEAD_STR_T'].iloc[count] += int(h[1])
                fi['BODY_STR_L'].iloc[count] += int(b[0])
                fi['BODY_STR_T'].iloc[count] += int(b[1])
                fi['LEG_STR_L'].iloc[count] += int(l[0])
                fi['LEG_STR_T'].iloc[count] += int(l[1])
                fi['DISTANCE_STR_L'].iloc[count] += int(d[0])
                fi['DISTANCE_STR_T'].iloc[count] += int(d[1])
                fi['CLINCH_STR_L'].iloc[count] += int(c[0])
                fi['CLINCH_STR_T'].iloc[count] += int(c[1])
                fi['GROUND_STR_L'].iloc[count] += int(g[0])
                fi['GROUND_STR_T'].iloc[count] += int(g[1])
                
                # Defense 
                hd = a[2].split(' of ')
                bd = a[4].split(' of ')
                ld = a[6].split(' of ')
                dd = a[8].split(' of ')
                cd = a[10].split(' of ')
                gd = a[12].split(' of ')
            
            
                
               
                fi['EATEN_HEAD_STR_L'].iloc[count] += int(hd[0])
                fi['EATEN_HEAD_STR_T'].iloc[count] += int(hd[1])
                fi['EATEN_BODY_STR_L'].iloc[count] += int(bd[0])
                fi['EATEN_BODY_STR_T'].iloc[count] += int(bd[1])
                fi['EATEN_LEG_STR_L'].iloc[count] += int(ld[0])
                fi['EATEN_LEG_STR_T'].iloc[count] += int(ld[1])
                fi['EATEN_DISTANCE_STR_L'].iloc[count] += int(dd[0])
                fi['EATEN_DISTANCE_STR_T'].iloc[count] += int(dd[1])
                fi['EATEN_CLINCH_STR_L'].iloc[count] += int(cd[0])
                fi['EATEN_CLINCH_STR_T'].iloc[count] += int(cd[1])
                fi['EATEN_GROUND_STR_L'].iloc[count] += int(gd[0])
                fi['EATEN_GROUND_STR_T'].iloc[count] += int(gd[1])
        
            
    count += 1
        
    

In [27]:
# Fighters area of attack percentages

fi['HEAD_PER'] = round(fi['HEAD_STR_L'] / fi['SIG_LAND'] * 100,2)
fi['BODY_PER'] = round(fi['BODY_STR_L'] / fi['SIG_LAND'] * 100,2)
fi['LEG_PER'] = round(fi['LEG_STR_L'] / fi['SIG_LAND'] * 100,2)


fi['DIST_PER'] = round(fi['DISTANCE_STR_L'] / fi['SIG_LAND'] * 100,2)
fi['CLINCH_PER'] = round(fi['CLINCH_STR_L'] / fi['SIG_LAND'] * 100,2)
fi['GROUND_PER'] = round(fi['GROUND_STR_L'] / fi['SIG_LAND'] * 100,2)


In [ ]:
#determining loss methods for all fighters

fi['KO_LOSS'] = 0
fi['SUB_LOSS'] = 0
fi['DEC_LOSS'] = 0

count = 0 

for i in fi['Full']:
    for a in advanced[['LOSER','FINISH',"WINNER"]].values:
        if name_check(i,a[0]):
            if 'Decision' in a[1]:
                fi['DEC_LOSS'].iloc[count] += 1
            elif 'KO' in a[1]:
                fi['KO_LOSS'].iloc[count] += 1
            
            elif 'Sub' in a[1]:
                fi['SUB_LOSS'].iloc[count] += 1
                print(i,a)
    count += 1

This concludes the fighter details database, here is a brief look at the final product.

In [29]:
fi.head(10)

,index,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,Full,UFC_W,UFC_L,UFC_D,UFC_NC,FIGHTS,SIG_LAND,SIG_THROWN,TOTAL_LAND,TOTAL_THROWN,TOTAL_EATEN_L,TOTAL_EATEN_T,SIG_EATEN_L,SIG_EATEN_T,SIG_PER,TOTAL_PER,TD_L,TD_A,SUB_A,REV,TD_PERC,KOs,SUBS,DEC,RD1_STRIKES_T,RD1_STRIKES_L,RD2_STRIKES_T,RD2_STRIKES_L,RD3_STRIKES_T,RD3_STRIKES_L,RD4_STRIKES_T,RD4_STRIKES_L,RD5_STRIKES_T,RD5_STRIKES_L,EATEN_RD1_STRIKES_T,EATEN_RD1_STRIKES_L,EATEN_RD2_STRIKES_T,EATEN_RD2_STRIKES_L,EATEN_RD3_STRIKES_T,EATEN_RD3_STRIKES_L,EATEN_RD4_STRIKES_T,EATEN_RD4_STRIKES_L,EATEN_RD5_STRIKES_T,EATEN_RD5_STRIKES_L,FIGHT_TIME,SPLIT_DEC,SPLIT_W,SPLIT_L,MAJ_W,MAJ_L,MAJ_DEC,RD1_STR_PERC,RD2_STR_PERC,RD3_STR_PERC,RD4_STR_PERC,RD5_STR_PERC,SIG_PER_MIN,TOTAL_PER_MIN,HEAD_STR_L,HEAD_STR_T,BODY_STR_L,BODY_STR_T,LEG_STR_L,LEG_STR_T,DISTANCE_STR_L,DISTANCE_STR_T,CLINCH_STR_L,CLINCH_STR_T,GROUND_STR_L,GROUND_STR_T,EATEN_HEAD_STR_L,EATEN_HEAD_STR_T,EATEN_BODY_STR_L,EATEN_BODY_STR_T,EATEN_LEG_STR_L,EATEN_LEG_STR_T,EATEN_DISTANCE_STR_L,EATEN_DISTANCE_STR_T,EATEN_CLINCH_STR_L,EATEN_CLINCH_STR_T,EATEN_GROUND_STR_L,EATEN_GROUND_STR_T,HEAD_PER,BODY_PER,LEG_PER,DIST_PER,CLINCH_PER,GROUND_PER,KO_LOSS,SUB_LOSS,DEC_LOSS
0,2,Danny,Abbadi,The Assassin,"5' 11""",155,--,Orthodox,4.0,6.0,0.0,NaN,Danny Abbadi,0,2,0,0,2,59,155,85,185,126,241,79,188,38.06,45.95,0,0,0,0,NaN,0,0,0,55,33,52,22,78,30,0,0,0,0,100,76,80,32,61,18,0,0,0,0,17.56,1,0,1,0,0,0,0.60,0.42,0.38,NaN,NaN,3.36,4.84,51,145,8,10,0,0,42,127,14,24,3,4,59,166,17,19,3,3,46,133,17,23,16,32,86.44,13.56,0.00,71.19,23.73,5.08,0,1,1
1,3,David,Abbott,Tank,"6' 0""",265,--,Switch,10.0,15.0,0.0,NaN,David Abbott,8,10,0,0,18,156,351,311,533,654,781,209,308,44.44,58.35,10,16,4,10,62.50,5,2,1,490,283,43,28,0,0,0,0,0,0,731,617,50,37,0,0,0,0,0,0,48.98,0,0,0,0,0,0,0.58,0.65,NaN,NaN,NaN,3.18,6.35,148,342,8,9,0,0,54,156,46,92,56,103,114,207,57,61,38,40,62,121,75,98,72,89,94.87,5.13,0.00,34.62,29.49,35.90,3,5,2
2,4,Shamil,Abdurakhimov,Abrek,"6' 3""",235,"76.0""",Orthodox,20.0,5.0,0.0,NaN,Shamil Abdurakhimov,5,3,0,0,8,210,469,341,613,269,584,210,508,44.78,55.63,7,29,1,4,24.14,2,0,3,252,133,203,129,149,76,9,3,0,0,261,118,151,71,124,52,48,28,0,0,84.60,1,1,0,0,0,0,0.53,0.64,0.51,0.33,NaN,2.48,4.03,131,354,37,58,42,57,129,347,59,91,22,31,138,412,42,61,30,35,93,323,53,91,64,94,62.38,17.62,20.00,61.43,28.10,10.48,3,0,0
3,6,Daichi,Abe,NaN,"5' 11""",170,"71.0""",Orthodox,6.0,2.0,0.0,NaN,Daichi Abe,1,2,0,0,3,171,508,175,515,203,470,202,469,33.66,33.98,1,2,0,1,50.00,0,0,1,218,79,151,47,146,49,0,0,0,0,144,60,168,74,158,69,0,0,0,0,45.00,0,0,0,0,0,0,0.36,0.31,0.34,NaN,NaN,3.80,3.89,102,404,12,27,57,77,146,470,11,20,14,18,93,312,28,49,81,108,196,456,3,8,3,5,59.65,7.02,33.33,85.38,6.43,8.19,0,0,2
4,7,Papy,Abedi,Makambo,"5' 11""",185,--,Southpaw,10.0,4.0,0.0,NaN,Papy Abedi,1,3,0,0,4,97,176,130,212,221,332,109,212,55.11,61.32,8,14,3,7,57.14,0,0,1,120,67,67,42,25,21,0,0,0,0,149,87,104,73,79,61,0,0,0,0,33.97,1,1,0,0,0,0,0.56,0.63,0.84,NaN,NaN,2.86,3.83,42,104,46,55,9,17,36,95,34,43,27,38,67,161,30,33,12,18,42,112,49,76,18,24,43.30,47.42,9.28,37.11,35.05,27.84,1,2,0
5,8,Ricardo,Abreu,Demente,"5' 11""",185,--,Orthodox,5.0,1.0,0.0,NaN,Ricardo Abreu,1,1,0,0,2,80,252,112,285,104,288,84,263,31.75,39.30,3,7,1,4,42.86,0,1,0,136,42,85,48,64,22,0,0,0,0,127,43,76,26,85,35,0,0,0,0,21.06,1,0,1,0,0,0,0.31,0.56,0.34,NaN,NaN,3.80,5.32,65,229,10,14,5,9,58,216,13,24,9,12,42,198,20,34,22,31,78,244,6,19,0,0,81.25,12.50,6.25,72.50,16.25,11.25,0,0,1
6,9,Klidson,Abreu,White Bear,"6' 0""",205,"74.0""",Orthodox,15.0,4.0,0.0,NaN,Klidson Abreu,1,3,0,0,4,96,236,145,290,229,406,136,306,40.68,50.00,2,10,0,1,20.00,0,0,1,98,53,85,55,107,37,0,0,0,0,161,89,107,56,138,84,0,0,0,0,46.51,1,0,1,0,0,0,0.54,0.65,0.35,NaN,NaN,2.06,3.12,38,151,20,44,38,41,86,220,10,15,0,1,53,186,38,59,45,61,112,276,12,15,12,15,39.58,20.83,39.58,89.58,10.42,0.00,1,0,2
7,11,Scott,Adams,NaN,"6' 0""",225,--,Southpaw,8.0,1.0,0.0,NaN,Scott Adams,1,0,0,0,1,0,0,10,11,3,3,0,0,NaN,90.91,1,1,4,0,100.00,0,1,0,11,10,0,0,0,0,0,0,0,0,3,3,0

The next step is to add the new results to the current working Fighter Profile database.

In [5]:
con = psycopg2.connect("Connection String")
print("Database opened successfully")
print()
cur = con.cursor()

cur.execute('SELECT * from "Fighter_Profile"')
tupples = cur.fetchall()
headers = [i[0] for i in cur.description]
Fighter_Profile = pd.DataFrame(tupples, columns=headers)

Fighter_Profile.tail(3) 

Database opened successfully



,Unnamed: 0,index,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,Full,UFC_W,UFC_L,UFC_D,UFC_NC,FIGHTS,SIG_LAND,SIG_THROWN,TOTAL_LAND,TOTAL_THROWN,TOTAL_EATEN_L,TOTAL_EATEN_T,SIG_EATEN_L,SIG_EATEN_T,SIG_PER,TOTAL_PER,TD_L,TD_A,SUB_A,REV,TD_PERC,KOs,SUBS,DEC,RD1_STRIKES_T,RD1_STRIKES_L,RD2_STRIKES_T,RD2_STRIKES_L,RD3_STRIKES_T,RD3_STRIKES_L,RD4_STRIKES_T,RD4_STRIKES_L,RD5_STRIKES_T,RD5_STRIKES_L,EATEN_RD1_STRIKES_T,EATEN_RD1_STRIKES_L,EATEN_RD2_STRIKES_T,EATEN_RD2_STRIKES_L,EATEN_RD3_STRIKES_T,EATEN_RD3_STRIKES_L,EATEN_RD4_STRIKES_T,EATEN_RD4_STRIKES_L,EATEN_RD5_STRIKES_T,EATEN_RD5_STRIKES_L,FIGHT_TIME,SPLIT_DEC,SPLIT_W,SPLIT_L,MAJ_W,MAJ_L,MAJ_DEC,RD1_STR_PERC,RD2_STR_PERC,RD3_STR_PERC,RD4_STR_PERC,RD5_STR_PERC,SIG_PER_MIN,TOTAL_PER_MIN,HEAD_STR_L,HEAD_STR_T,BODY_STR_L,BODY_STR_T,LEG_STR_L,LEG_STR_T,DISTANCE_STR_L,DISTANCE_STR_T,CLINCH_STR_L,CLINCH_STR_T,GROUND_STR_L,GROUND_STR_T,EATEN_HEAD_STR_L,EATEN_HEAD_STR_T,EATEN_BODY_STR_L,EATEN_BODY_STR_T,EATEN_LEG_STR_L,EATEN_LEG_STR_T,EATEN_DISTANCE_STR_L,EATEN_DISTANCE_STR_T,EATEN_CLINCH_STR_L,EATEN_CLINCH_STR_T,EATEN_GROUND_STR_L,EATEN_GROUND_STR_T,HEAD_PER,BODY_PER,LEG_PER,DIST_PER,CLINCH_PER,GROUND_PER,KO_LOSS,SUB_LOSS,DEC_LOSS
2061,1890,130,Marcin,Tybura,Tybur,"6' 3""",249,"78.0""",Orthodox,21.0,6.0,0.0,0.0,Marcin Tybura,8,5,0,0,13,615,1221,1108,1833,721,1446,540,1234,50.37,60.45,18,40,1,21,45.0,3,0,5,649,377,514,310,543,360,32,18,95,43,588,272,449,230,279,139,57,42,73,38,164.52,0,0,0,0,0,0,0.58,0.60,0.66,0.56,0.45,3.74,6.73,419,965,103,149,93,107,396,910,65,96,154,215,373,1037,103,122,64,75,415,1063,98,134,27,37,68.13,16.75,15.12,64.39,10.57,25.04,3,0,2
2062,1923,40,Marlon,Vera,Chito,"5' 8""",135,"70.0""",Switch,16.0,7.0,1.0,0.0,Marlon Vera,10,6,0,0,16,629,1281,778,1458,960,1731,682,1421,49.10,53.36,9,24,13,14,37.5,5,4,1,484,259,507,267,467,252,0,0,0,0,596,349,655,387,480,224,0,0,0,0,170.74,0,0,0,0,0,0,0.54,0.53,0.54,0.00,0.00,3.68,4.56,300,808,158,261,171,212,442,1020,97,128,90,133,412,1025,157,237,113,159,483,1139,87,127,112,155,47.69,25.12,27.19,70.27,15.42,14.31,0,0,6
2063,1998,88,Khaos,Williams,The OxFighter,"6' 0""",170,"77.0""",Orthodox,11.0,2.0,0.0,0.0,Khaos Williams,2,1,0,0,3,70,154,77,161,59,100,47,85,45.45,47.83,0,1,0,0,0.0,2,0,0,58,27,56,28,47,22,0,0,0,0,26,14,32,19,42,26,0,0,0,0,15.57,0,0,0,0,0,0,0.47,0.50,0.47,0.00,0.00,4.50,4.95,31,93,13,23,26,38,56,133,11,17,3,4,25,56,20,27,2,2,36,73,9,10,2,2,44.29,18.57,37.14,80.00,15.71,4.29,0,0,1


In [6]:
event = advanced.tail(2)["EVENT"].max()
print(str(event))

data = advanced[advanced["EVENT"] == event]
data.head(3)


UFC Fight Night: Thompson vs. Neal


,Unnamed: 0,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,REV_RED,REV_BLUE,CONTROL_RED,CONTROL_BLUE,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND1_CONTROL_RED,RND1_CONTROL_BLUE,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND2_CONTROL_RED,RND2_CONTROL_BLUE,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND3_CONTROL_RED,RND3_CONTROL_BLUE,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND4_CONTROL_RED,RND4_CONTROL_BLUE,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_REV_RED,RND5_REV_BLUE,RND5_CONTROL_RED,RND5_CONTROL_BLUE,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE,LOSER
5912,3,Marlon Moraes,Rob Font,Bantamweight Bout,UFC Fight Night: Thompson vs. Neal,"December 19, 2020","Las Vegas, Nevada, USA",0,KO/TKO,1,3:47,3 Rnd (5-5-5),Marc Goddard,Punches to Head On Ground,Rob Font,Non-Title Bout,PERF,0.0,1.0,9 of 12,30 of 37,75%,81%,12 of 16,33 of 41,2 of 3,0 of 0,66%,---,0.0,0.0,0.0,1.0,2:31,0:38,0.0,1.0,9 of 12,30 of 37,75%,81%

In [7]:
fighter_list = []
for i in data["FIGHTER_RED"]:
    fighter_list.append(i)

for i in data["FIGHER_BLUE"]:
    fighter_list.append(i)

print(fighter_list, len(fighter_list))

['Marlon Moraes', 'Stephen Thompson', 'Jose Aldo', 'Michel Pereira', 'Marcin Tybura', 'Anthony Pettis', 'Sijara Eubanks', 'Deron Winn', 'Gillian Robertson', 'Tafon Nchukwi', 'Christos Giagos', 'Jimmy Flick', 'Rob Font', 'Geoff Neal', 'Marlon Vera', 'Khaos Williams', 'Greg Hardy', 'Alex Morono', 'Pannie Kianzad', 'Antonio Arroyo', 'Taila Santos', 'Jamie Pickett', 'Carlton Minus', 'Cody Durden'] 24


In [88]:
update = fi[fi["Full"].isin(fighter_list)]
update.head(3)

,index,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,Full,UFC_W,UFC_L,UFC_D,UFC_NC,FIGHTS,SIG_LAND,SIG_THROWN,TOTAL_LAND,TOTAL_THROWN,TOTAL_EATEN_L,TOTAL_EATEN_T,SIG_EATEN_L,SIG_EATEN_T,SIG_PER,TOTAL_PER,TD_L,TD_A,SUB_A,REV,TD_PERC,KOs,SUBS,DEC,RD1_STRIKES_T,RD1_STRIKES_L,RD2_STRIKES_T,RD2_STRIKES_L,RD3_STRIKES_T,RD3_STRIKES_L,RD4_STRIKES_T,RD4_STRIKES_L,RD5_STRIKES_T,RD5_STRIKES_L,EATEN_RD1_STRIKES_T,EATEN_RD1_STRIKES_L,EATEN_RD2_STRIKES_T,EATEN_RD2_STRIKES_L,EATEN_RD3_STRIKES_T,EATEN_RD3_STRIKES_L,EATEN_RD4_STRIKES_T,EATEN_RD4_STRIKES_L,EATEN_RD5_STRIKES_T,EATEN_RD5_STRIKES_L,FIGHT_TIME,SPLIT_DEC,SPLIT_W,SPLIT_L,MAJ_W,MAJ_L,MAJ_DEC,RD1_STR_PERC,RD2_STR_PERC,RD3_STR_PERC,RD4_STR_PERC,RD5_STR_PERC,SIG_PER_MIN,TOTAL_PER_MIN,HEAD_STR_L,HEAD_STR_T,BODY_STR_L,BODY_STR_T,LEG_STR_L,LEG_STR_T,DISTANCE_STR_L,DISTANCE_STR_T,CLINCH_STR_L,CLINCH_STR_T,GROUND_STR_L,GROUND_STR_T,EATEN_HEAD_STR_L,EATEN_HEAD_STR_T,EATEN_BODY_STR_L,EATEN_BODY_STR_T,EATEN_LEG_STR_L,EATEN_LEG_STR_T,EATEN_DISTANCE_STR_L,EATEN_DISTANCE_STR_T,EATEN_CLINCH_STR_L,EATEN_CLINCH_STR_T,EATEN_GROUND_STR_L,EATEN_GROUND_STR_T,HEAD_PER,BODY_PER,LEG_PER,DIST_PER,CLINCH_PER,GROUND_PER,KO_LOSS,SUB_LOSS,DEC_LOSS
34,49,Jose,Aldo,NaN,"5' 7""",135,"70.0""",Orthodox,29.0,7.0,0.0,NaN,Jose Aldo,11,6,0,0,17,920,2118,1065,2274,1464,3147,1119,2752,43.44,46.83,12,22,3,12,54.55,4,0,7,629,308,598,299,567,260,293,122,187,76,700,281,737,282,903,484,358,169,449,248,278.23,1,0,1,0,0,0,0.49,0.50,0.46,0.42,0.41,3.31,3.83,641,1721,163,258,116,139,813,1961,44,54,63,103,701,2148,203,309,215,295,833,2377,115,144,171,231,69.67,17.72,12.61,88.37,4.78,6.85,4,0,2
87,139,Antonio,Arroyo,NaN,"6' 3""",185,"73.0""",Orthodox,9.0,4.0,0.0,NaN,Antonio Arroyo,0,2,0,0,2,74,109,117,154,149,217,42,80,67.89,75.97,1,1,0,0,100.00,0,0,0,64,50,41,30,49,37,0,0,0,0,48,29,84,53,85,67,0,0,0,0,30.00,0,0,0,0,0,0,0.78,0.73,0.76,NaN,NaN,2.47,3.90,49,80,16,19,9,10,64,96,0,0,10,13,22,56,13,17,7,7,38,75,3,3,1,2,66.22,21.62,12.16,86.49,0.00,13.51,0,0,2
480,160,Cody,Durden,NaN,"5' 7""",125,"67.0""",Southpaw,11.0,3.0,1.0,NaN,Cody Durden,0,1,1,0,2,57,117,125,232,80,131,72,123,48.72,53.88,4,15,1,1,26.67,0,0,0,149,87,44,21,39,17,0,0,0,0,18,10,51,31,62,39,0,0,0,0,18.18,0,0,0,0,0,0,0.58,0.48,0.44,NaN,NaN,3.14,6.88,37,89,9,16,11,12,51,108,2,3,4,6,32,66,9,16,31,41,65,112,3,4,4,7,64.91,15.79,19.30,89.47,3.51,7.02,0,1,0


In [89]:
update = update[["Stance","Full"]].merge(update.iloc[:,8:])
list1= list(update.columns)
list1.remove("Full")
list1.append("Full")
update = update.reindex(list1, axis="columns")
update.fillna(value = 0,inplace = True)
update.head(3)

,Stance,W,L,D,Belt,UFC_W,UFC_L,UFC_D,UFC_NC,FIGHTS,SIG_LAND,SIG_THROWN,TOTAL_LAND,TOTAL_THROWN,TOTAL_EATEN_L,TOTAL_EATEN_T,SIG_EATEN_L,SIG_EATEN_T,SIG_PER,TOTAL_PER,TD_L,TD_A,SUB_A,REV,TD_PERC,KOs,SUBS,DEC,RD1_STRIKES_T,RD1_STRIKES_L,RD2_STRIKES_T,RD2_STRIKES_L,RD3_STRIKES_T,RD3_STRIKES_L,RD4_STRIKES_T,RD4_STRIKES_L,RD5_STRIKES_T,RD5_STRIKES_L,EATEN_RD1_STRIKES_T,EATEN_RD1_STRIKES_L,EATEN_RD2_STRIKES_T,EATEN_RD2_STRIKES_L,EATEN_RD3_STRIKES_T,EATEN_RD3_STRIKES_L,EATEN_RD4_STRIKES_T,EATEN_RD4_STRIKES_L,EATEN_RD5_STRIKES_T,EATEN_RD5_STRIKES_L,FIGHT_TIME,SPLIT_DEC,SPLIT_W,SPLIT_L,MAJ_W,MAJ_L,MAJ_DEC,RD1_STR_PERC,RD2_STR_PERC,RD3_STR_PERC,RD4_STR_PERC,RD5_STR_PERC,SIG_PER_MIN,TOTAL_PER_MIN,HEAD_STR_L,HEAD_STR_T,BODY_STR_L,BODY_STR_T,LEG_STR_L,LEG_STR_T,DISTANCE_STR_L,DISTANCE_STR_T,CLINCH_STR_L,CLINCH_STR_T,GROUND_STR_L,GROUND_STR_T,EATEN_HEAD_STR_L,EATEN_HEAD_STR_T,EATEN_BODY_STR_L,EATEN_BODY_STR_T,EATEN_LEG_STR_L,EATEN_LEG_STR_T,EATEN_DISTANCE_STR_L,EATEN_DISTANCE_STR_T,EATEN_CLINCH_STR_L,EATEN_CLINCH_STR_T,EATEN_GROUND_STR_L,EATEN_GROUND_STR_T,HEAD_PER,BODY_PER,LEG_PER,DIST_PER,CLINCH_PER,GROUND_PER,KO_LOSS,SUB_LOSS,DEC_LOSS,Full
0,Orthodox,29.0,7.0,0.0,0.0,11,6,0,0,17,920,2118,1065,2274,1464,3147,1119,2752,43.44,46.83,12,22,3,12,54.55,4,0,7,629,308,598,299,567,260,293,122,187,76,700,281,737,282,903,484,358,169,449,248,278.23,1,0,1,0,0,0,0.49,0.50,0.46,0.42,0.41,3.31,3.83,641,1721,163,258,116,139,813,1961,44,54,63,103,701,2148,203,309,215,295,833,2377,115,144,171,231,69.67,17.72,12.61,88.37,4.78,6.85,4,0,2,Jose Aldo
1,Orthodox,9.0,4.0,0.0,0.0,0,2,0,0,2,74,109,117,154,149,217,42,80,67.89,75.97,1,1,0,0,100.00,0,0,0,64,50,41,30,49,37,0,0,0,0,48,29,84,53,85,67,0,0,0,0,30.00,0,0,0,0,0,0,0.78,0.73,0.76,0.00,0.00,2.47,3.90,49,80,16,19,9,10,64,96,0,0,10,13,22,56,13,17,7,7,38,75,3,3,1,2,66.22,21.62,12.16,86.49,0.00,13.51,0,0,2,Antonio Arroyo
2,Southpaw,11.0,3.0,1.0,0.0,0,1,1,0,2,57,117,125,232,80,131,72,123,48.72,53.88,4,15,1,1,26.67,0,0,0,149,87,44,21,39,17,0,0,0,0,18,10,51,31,62,39,0,0,0,0,18.18,0,0,0,0,0,0,0.58,0.48,0.44,0.00,0.00,3.14,6.88,37,89,9,16,11,12,51,108,2,3,4,6,32,66,9,16,31,41,65,112,3,4,4,7,64.91,15.79,19.30,89.47,3.51,7.02,0,1,0,Cody Durden


In [ ]:
SQL = '''
UPDATE "Fighter_Profile"
SET "Stance" = '{}',
    "W" = {},
    "L" = {},
    "D" = {},
    "Belt" = '{}',
    "UFC_W" = {},
    "UFC_L" = {},
    "UFC_D" = {},
    "UFC_NC" = {},
    "FIGHTS" = {},
    "SIG_LAND" = {},
    "SIG_THROWN" = {},
    "TOTAL_LAND" = {},
    "TOTAL_THROWN" = {},
    "TOTAL_EATEN_L" = {},
    "TOTAL_EATEN_T" = {},
    "SIG_EATEN_L" = {},
    "SIG_EATEN_T" = {},
    "SIG_PER" = {},
    "TOTAL_PER" = {},
    "TD_L" = {},
    "TD_A" = {},
    "SUB_A" = {},
    "REV" = {},
    "TD_PERC" = {},
    "KOs" = {},
    "SUBS" = {},
    "DEC" = {}, 
    "RD1_STRIKES_T" = {},
    "RD1_STRIKES_L" = {},
    "RD2_STRIKES_T" = {},
    "RD2_STRIKES_L" = {},
    "RD3_STRIKES_T" = {},
    "RD3_STRIKES_L" = {},
    "RD4_STRIKES_T" = {},
    "RD4_STRIKES_L" = {},
    "RD5_STRIKES_T" = {},
    "RD5_STRIKES_L" = {},
    "EATEN_RD1_STRIKES_T" = {},
    "EATEN_RD1_STRIKES_L" = {},
    "EATEN_RD2_STRIKES_T" = {},
    "EATEN_RD2_STRIKES_L" = {},
    "EATEN_RD3_STRIKES_T" = {},
    "EATEN_RD3_STRIKES_L" = {},
    "EATEN_RD4_STRIKES_T" = {},
    "EATEN_RD4_STRIKES_L" = {},
    "EATEN_RD5_STRIKES_T" = {},
    "EATEN_RD5_STRIKES_L" = {},
    "FIGHT_TIME" = {},
    "SPLIT_DEC" = {},
    "SPLIT_W" = {},
    "SPLIT_L" = {},
    "MAJ_W" = {},
    "MAJ_L" = {},
    "MAJ_DEC" = {},
    "RD1_STR_PERC" = {},
    "RD2_STR_PERC" = {},
    "RD3_STR_PERC" = {},
    "RD4_STR_PERC" = {},
    "RD5_STR_PERC" = {},
    "SIG_PER_MIN" = {},
    "TOTAL_PER_MIN" = {},
    "HEAD_STR_L" = {},
    "HEAD_STR_T" = {},
    "BODY_STR_L" = {},
    "BODY_STR_T" = {},
    "LEG_STR_L" = {},
    "LEG_STR_T" = {},
    "DISTANCE_STR_L" = {},
    "DISTANCE_STR_T" = {},
    "CLINCH_STR_L" = {},
    "CLINCH_STR_T" = {},
    "GROUND_STR_L" = {},
    "GROUND_STR_T" = {},
    "EATEN_HEAD_STR_L" = {},
    "EATEN_HEAD_STR_T" = {},
    "EATEN_BODY_STR_L" = {},
    "EATEN_BODY_STR_T" = {},
    "EATEN_LEG_STR_L" = {},
    "EATEN_LEG_STR_T" = {},
    "EATEN_DISTANCE_STR_L" = {},
    "EATEN_DISTANCE_STR_T" = {},
    "EATEN_CLINCH_STR_L" = {},
    "EATEN_CLINCH_STR_T" = {},
    "EATEN_GROUND_STR_L" = {},
    "EATEN_GROUND_STR_T" = {},
    "HEAD_PER" = {},
    "BODY_PER" = {},
    "LEG_PER" = {},
    "DIST_PER" = {},
    "CLINCH_PER" = {},
    "GROUND_PER" = {},
    "KO_LOSS" = {},
    "SUB_LOSS" = {},
    "DEC_LOSS" = {}
    
    WHERE "Full" = '{}'
    
'''



for i in update.values:
    temp = SQL.format(*list(i))
    cur.execute(temp)
    con.commit()

In [91]:
comp.head(3)

,Unnamed: 0,index,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,Full,UFC_W,UFC_L,UFC_D,UFC_NC,FIGHTS,SIG_LAND,SIG_THROWN,TOTAL_LAND,TOTAL_THROWN,TOTAL_EATEN_L,TOTAL_EATEN_T,SIG_EATEN_L,SIG_EATEN_T,SIG_PER,TOTAL_PER,TD_L,TD_A,SUB_A,REV,TD_PERC,KOs,SUBS,DEC,RD1_STRIKES_T,RD1_STRIKES_L,RD2_STRIKES_T,RD2_STRIKES_L,RD3_STRIKES_T,RD3_STRIKES_L,RD4_STRIKES_T,RD4_STRIKES_L,RD5_STRIKES_T,RD5_STRIKES_L,EATEN_RD1_STRIKES_T,EATEN_RD1_STRIKES_L,EATEN_RD2_STRIKES_T,EATEN_RD2_STRIKES_L,EATEN_RD3_STRIKES_T,EATEN_RD3_STRIKES_L,EATEN_RD4_STRIKES_T,EATEN_RD4_STRIKES_L,EATEN_RD5_STRIKES_T,EATEN_RD5_STRIKES_L,FIGHT_TIME,SPLIT_DEC,SPLIT_W,SPLIT_L,MAJ_W,MAJ_L,MAJ_DEC,RD1_STR_PERC,RD2_STR_PERC,RD3_STR_PERC,RD4_STR_PERC,RD5_STR_PERC,SIG_PER_MIN,TOTAL_PER_MIN,HEAD_STR_L,HEAD_STR_T,BODY_STR_L,BODY_STR_T,LEG_STR_L,LEG_STR_T,DISTANCE_STR_L,DISTANCE_STR_T,CLINCH_STR_L,CLINCH_STR_T,GROUND_STR_L,GROUND_STR_T,EATEN_HEAD_STR_L,EATEN_HEAD_STR_T,EATEN_BODY_STR_L,EATEN_BODY_STR_T,EATEN_LEG_STR_L,EATEN_LEG_STR_T,EATEN_DISTANCE_STR_L,EATEN_DISTANCE_STR_T,EATEN_CLINCH_STR_L,EATEN_CLINCH_STR_T,EATEN_GROUND_STR_L,EATEN_GROUND_STR_T,HEAD_PER,BODY_PER,LEG_PER,DIST_PER,CLINCH_PER,GROUND_PER,KO_LOSS,SUB_LOSS,DEC_LOSS
86,87,139,Antonio,Arroyo,None,"6' 3""",185,"73.0""",Orthodox,9.0,3.0,0.0,NaN,Antonio Arroyo,0,1,0,0,1,29,47,47,67,85,128,22,42,61.70,70.15,1,1,0,0,100.00,0,0,0,27,17,22,16,18,14,0,0,0,0,33,20,45,23,50,42,0,0,0,0,15.0,0,0,0,0,0,0,0.63,0.73,0.78,NaN,NaN,1.93,3.13,20,35,5,7,4,5,21,36,0,0,8,11,12,30,8,10,2,2,20,40,1,1,1,1,68.97,17.24,13.79,72.41,0.00,27.59,0,0,1
479,480,160,Cody,Durden,None,"5' 7""",125,"67.0""",Southpaw,11.0,2.0,1.0,NaN,Cody Durden,0,0,1,0,1,36,85,95,189,73,116,66,109,42.35,50.26,2,13,1,1,15.38,0,0,0,106,57,44,21,39,17,0,0,0,0,3,3,51,31,62,39,0,0,0,0,15.0,0,0,0,0,0,0,0.54,0.48,0.44,NaN,NaN,2.40,6.33,20,61,6,13,10,11,34,80,0,1,2,4,28,59,9,15,29,35,59,98,3,4,4,7,55.56,16.67,27.78,94.44,0.00,5.56,0,0,0
527,528,69,Sijara,Eubanks,Sarj,"5' 4""",135,"67.0""",Orthodox,7.0,5.0,0.0,NaN,Sijara Eubanks,4,3,0,0,7,493,1148,634,1310,549,1120,351,893,42.94,48.40,13,28,3,8,46.43,0,0,4,418,194,350,169,542,271,0,0,0,0,343,149,409,232,368,168,0,0,0,0,105.0,0,0,0,0,0,0,0.46,0.48,0.50,NaN,NaN,4.70,6.04,362,970,91,134,40,44,373,968,33,48,87,132,274,776,49,81,28,36,298,816,32,50,21,27,73.43,18.46,8.11,75.66,6.69,17.65,0,0,3


In [8]:
con = psycopg2.connect("Connection String")
print("Database opened successfully")
print()
cur = con.cursor()

cur.execute('SELECT * from "Fighter_Profile"')
tupples = cur.fetchall()
headers = [i[0] for i in cur.description]
Fighter_Profile = pd.DataFrame(tupples, columns=headers)

Fighter_Profile[Fighter_Profile["Full"].isin(fighter_list)].head(3)

Database opened successfully



,Unnamed: 0,index,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,Full,UFC_W,UFC_L,UFC_D,UFC_NC,FIGHTS,SIG_LAND,SIG_THROWN,TOTAL_LAND,TOTAL_THROWN,TOTAL_EATEN_L,TOTAL_EATEN_T,SIG_EATEN_L,SIG_EATEN_T,SIG_PER,TOTAL_PER,TD_L,TD_A,SUB_A,REV,TD_PERC,KOs,SUBS,DEC,RD1_STRIKES_T,RD1_STRIKES_L,RD2_STRIKES_T,RD2_STRIKES_L,RD3_STRIKES_T,RD3_STRIKES_L,RD4_STRIKES_T,RD4_STRIKES_L,RD5_STRIKES_T,RD5_STRIKES_L,EATEN_RD1_STRIKES_T,EATEN_RD1_STRIKES_L,EATEN_RD2_STRIKES_T,EATEN_RD2_STRIKES_L,EATEN_RD3_STRIKES_T,EATEN_RD3_STRIKES_L,EATEN_RD4_STRIKES_T,EATEN_RD4_STRIKES_L,EATEN_RD5_STRIKES_T,EATEN_RD5_STRIKES_L,FIGHT_TIME,SPLIT_DEC,SPLIT_W,SPLIT_L,MAJ_W,MAJ_L,MAJ_DEC,RD1_STR_PERC,RD2_STR_PERC,RD3_STR_PERC,RD4_STR_PERC,RD5_STR_PERC,SIG_PER_MIN,TOTAL_PER_MIN,HEAD_STR_L,HEAD_STR_T,BODY_STR_L,BODY_STR_T,LEG_STR_L,LEG_STR_T,DISTANCE_STR_L,DISTANCE_STR_T,CLINCH_STR_L,CLINCH_STR_T,GROUND_STR_L,GROUND_STR_T,EATEN_HEAD_STR_L,EATEN_HEAD_STR_T,EATEN_BODY_STR_L,EATEN_BODY_STR_T,EATEN_LEG_STR_L,EATEN_LEG_STR_T,EATEN_DISTANCE_STR_L,EATEN_DISTANCE_STR_T,EATEN_CLINCH_STR_L,EATEN_CLINCH_STR_T,EATEN_GROUND_STR_L,EATEN_GROUND_STR_T,HEAD_PER,BODY_PER,LEG_PER,DIST_PER,CLINCH_PER,GROUND_PER,KO_LOSS,SUB_LOSS,DEC_LOSS
1986,2004,97,Deron,Winn,None,"5' 6""",185,"70.0""",Orthodox,7.0,2.0,0.0,0.0,Deron Winn,2,2,0,0,4,271,569,354,667,341,639,296,592,47.63,53.07,19,36,0,1,52.78,0,0,2,188,104,262,134,217,116,0,0,0,0,200,113,217,124,222,104,0,0,0,0,57.13,1,0,1,0,0,0,0.55,0.51,0.53,0.00,0.00,4.74,6.20,197,484,56,67,18,18,213,491,47,66,11,12,239,532,51,54,6,6,229,511,53,65,14,16,72.69,20.66,6.64,78.60,17.34,4.06,0,1,1
2044,34,49,Jose,Aldo,None,"5' 7""",135,"70.0""",Orthodox,29.0,7.0,0.0,0.0,Jose Aldo,11,6,0,0,17,920,2118,1065,2274,1464,3147,1119,2752,43.44,46.83,12,22,3,12,54.55,4,0,7,629,308,598,299,567,260,293,122,187,76,700,281,737,282,903,484,358,169,449,248,278.23,1,0,1,0,0,0,0.49,0.50,0.46,0.42,0.41,3.31,3.83,641,1721,163,258,116,139,813,1961,44,54,63,103,701,2148,203,309,215,295,833,2377,115,144,171,231,69.67,17.72,12.61,88.37,4.78,6.85,4,0,2
2045,87,139,Antonio,Arroyo,None,"6' 3""",185,"73.0""",Orthodox,9.0,4.0,0.0,0.0,Antonio Arroyo,0,2,0,0,2,74,109,117,154,149,217,42,80,67.89,75.97,1,1,0,0,100.00,0,0,0,64,50,41,30,49,37,0,0,0,0,48,29,84,53,85,67,0,0,0,0,30.00,0,0,0,0,0,0,0.78,0.73,0.76,0.00,0.00,2.47,3.90,49,80,16,19,9,10,64,96,0,0,10,13,22,56,13,17,7,7,38,75,3,3,1,2,66.22,21.62,12.16,86.49,0.00,13.51,0,0,2


As you can see the database is now updated with the new correct values.

In [ ]:
# added to csv as an extra backup
fi.to_csv(r'C:\Users\brock\OneDrive\Documents\UFC_Fighter_Stats.csv')